### Using Boruta on the Madalon Data Set
Author: [Mike Bernico](mike.bernico@gmail.com)

This example demonstrates using Boruta to find all relevant features in the Madalon dataset, which is an artificial dataset used in NIPS2003 and cited in the [Boruta paper](https://www.jstatsoft.org/article/view/v036i11/v36i11.pdf)

This dataset has 2000 observations and 500 features.  We will use Boruta to identify the features that are relevant to the classification task.





In [1]:
# Installation
#!pip install boruta

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from boruta_py import BorutaPy
from multiprocessing import cpu_count

In [3]:
def load_data():
    # URLS for dataset via UCI
    train_data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/madelon/MADELON/madelon_train.data'
    train_label_url='https://archive.ics.uci.edu/ml/machine-learning-databases/madelon/MADELON/madelon_train.labels'
        
    
    X_data = pd.read_csv(train_data_url, sep=" ", header=None)
    y_data = pd.read_csv(train_label_url, sep=" ", header=None)
    data = X_data.iloc[:,0:499]
    data['target'] = y_data[0] 
    return data

In [4]:
data = load_data()
data.shape

(2000, 500)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,target
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,-1
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,-1
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,-1
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,1
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,1


In [6]:
y=data.pop('target')
X=data.copy()

In [7]:
y.unique()

array([-1,  1])

Boruta conforms to the sklearn api and can be used in a Pipeline as well as on it's own. Here we will demonstrate stand alone operation.

First we will instantiate an estimator that Boruta will use.  Then we will instantiate a Boruta Object.

In [8]:
rf = RandomForestClassifier(n_jobs=int(cpu_count()/2), class_weight='balanced', max_depth=7)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', two_step=False,verbose=2, random_state=42)
# 計算を軽くするtwo_stepがない方が再現実験としてうまくいく

Once built, we can use this object to identify the relevant features in our dataset.

In [9]:
feat_selector.fit(X.values,y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	10 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	11 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	12 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	13 / 100
Confirmed: 	0
Tentative: 	499
Rejected: 	0

Iteration: 	14 / 100
Confirmed: 	19
Tentative: 	8
Rejected: 	472

Iteration: 	15 / 100
Confirmed: 	19
Tentative: 	8
Rejected: 	472

Iteration: 	16 / 100
Confirmed: 

BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=7, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=92, n_jobs=6, oob_score=False,
            random_state=<mtrand.RandomState object at 0x10a5725e8>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x10a5725e8>,
     two_step=False, verbose=2)

Boruta has confirmed only a few features as useful.   When our run ended, Boruta was undecided on 2 features.   '

We can interrogate .support_ to understand which features were selected.   .support_ returns an array of booleans that we can use to slice our feature matrix to include only relevant columns.   Of course, .transform can also be used, as expected in the scikit API.

In [10]:
# check selected features
print(feat_selector.support_)
#select the chosen features from our dataframe.
selected = X.iloc[:,feat_selector.support_]
print ("")
print ("Selected Feature Matrix Shape")
print (selected.shape)

[False False False False  True False False False False False  True False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False Fa

We can also interrogate the ranking of the unselected features with .ranking_

In [11]:
feat_selector.ranking_

array([437, 328, 140, 429,   1, 257, 249, 473, 322, 444,   1, 119, 129,
        70, 203, 279, 313, 146, 105, 361, 387, 365, 419, 259, 118,  58,
       114, 390,   1, 149, 445, 167,  47, 191, 398, 166, 246, 282,  43,
       464, 446, 189, 178, 129,  59, 327,  79, 311,   1,  49, 198, 196,
       219, 400, 111,  12,  11,  42, 289, 103,  41, 297, 395, 436,   1,
       184, 160, 201, 352, 420, 329, 201, 265,  21, 285, 146, 406,  96,
       330, 160, 269,  40, 282, 332, 135, 192, 430, 271, 359, 117, 477,
       246, 399, 348, 316, 127, 454, 188, 134, 204, 319, 422, 394, 228,
       221,   1,  74, 333, 306, 277, 384, 116, 421, 243, 228, 216,  36,
       199, 176,  37, 457, 267, 291, 384, 242, 267, 380, 108,   1, 112,
       166,  68, 345, 443, 260, 262,  13,  27, 425, 153,  57, 339, 402,
       396, 268, 351, 419,  56, 391,  33, 296, 318,  94,   1, 461, 151,
       139, 364, 214,  88, 347,  91, 376, 322,  72, 142, 469, 423, 472,
       366, 107, 173, 131, 459, 409, 100, 334,  82, 314, 350,  4